### **分类变量编码选择表**

| 模型 | 推荐方法 | 原因 | 代码示例 |
|------|---------|------|---------|
| **RandomForestRegressor** | ✅ One-Hot | 无 categorical_features 参数 | `pd.get_dummies()` |
| **HistGradientBoostingRegressor** | ✅ OrdinalEncoder + categorical_features | 原生支持，性能最佳 | `categorical_features=[0]` |
| **XGBoost** | ✅ LabelEncoder + enable_categorical | 需设置 enable_categorical=True | `enable_categorical=True` |
| **LightGBM** | ✅ 原始字符串 | 自动识别 | 直接传入字符串列 |
| **CatBoost** | ✅ 原始字符串 | 自动识别 | `cat_features=['col']` |
| **线性模型** | ✅ One-Hot | 需要线性可分 | `pd.get_dummies()` |

## 📊 1. 核心决策表：按数据集特征选择模型

### **表1：数据集特征 → 推荐模型**

| 数据集特征 | 样本量 | 特征类型 | 推荐模型（优先级排序） | 原因 |
|-----------|--------|---------|---------------------|------|
| **小数据集** | < 1,000 | 表格 | ① Ridge/Lasso ② RandomForest ③ SVM | 防止过拟合，需要强正则化 |
| **中等数据集** | 1K-100K | 表格 | ① **XGBoost/LightGBM** ② HistGradientBoosting ③ RandomForest | 最佳平衡点，树模型占优 |
| **大数据集** | 100K-1M | 表格 | ① **LightGBM** ② HistGradientBoosting ③ 神经网络 | 速度和性能兼顾 |
| **超大数据集** | > 1M | 表格 | ① **LightGBM** ② 神经网络(FastAI) ③ 在线学习(SGD) | 需要内存效率和并行化 |
| **高维稀疏** | 任意 | 稀疏特征 | ① **线性模型**(Ridge) ② LightGBM ③ 神经网络 | 避免维度灾难 |
| **低维密集** | 任意 | < 20特征 | ① **Gradient Boosting** ② RandomForest | 充分利用特征交互 |
| **时序数据** | 任意 | 时间序列 | ① **LSTM/GRU** ② XGBoost(lag特征) ③ ARIMA | 捕捉时间依赖 |
| **图像/非结构化** | > 10K | 图像/文本 | ① **神经网络**(CNN/Transformer) ② 迁移学习 | 需要表示学习 |
| **混合类型** | 任意 | 数值+分类 | ① **CatBoost** ② LightGBM ③ HistGradientBoosting | 原生支持分类变量 |

---

## 📈 2. 详细决策矩阵

### **表2：数据特征 vs 模型性能矩阵**

| 数据特征 | 线性模型 | RandomForest | XGBoost/LightGBM | HistGradientBoosting | 神经网络 | CatBoost |
|---------|---------|--------------|------------------|---------------------|----------|---------|
| **样本 < 1K** | ⭐⭐⭐⭐⭐ | ⭐⭐⭐ | ⭐⭐ | ⭐⭐ | ⭐ | ⭐⭐ |
| **样本 1K-10K** | ⭐⭐⭐ | ⭐⭐⭐⭐ | ⭐⭐⭐⭐⭐ | ⭐⭐⭐⭐ | ⭐⭐ | ⭐⭐⭐⭐⭐ |
| **样本 10K-100K** | ⭐⭐ | ⭐⭐⭐⭐ | ⭐⭐⭐⭐⭐ | ⭐⭐⭐⭐⭐ | ⭐⭐⭐ | ⭐⭐⭐⭐⭐ |
| **样本 > 100K** | ⭐⭐ | ⭐⭐⭐ | ⭐⭐⭐⭐⭐ | ⭐⭐⭐⭐⭐ | ⭐⭐⭐⭐ | ⭐⭐⭐⭐ |
| **高基数分类** | ⭐ | ⭐⭐ | ⭐⭐⭐ | ⭐⭐⭐ | ⭐⭐⭐⭐ | ⭐⭐⭐⭐⭐ |
| **缺失值多** | ⭐ | ⭐⭐⭐ | ⭐⭐⭐⭐⭐ | ⭐⭐⭐⭐⭐ | ⭐⭐ | ⭐⭐⭐⭐⭐ |
| **特征 < 10** | ⭐⭐⭐ | ⭐⭐⭐⭐ | ⭐⭐⭐⭐⭐ | ⭐⭐⭐⭐⭐ | ⭐⭐ | ⭐⭐⭐⭐⭐ |
| **特征 > 100** | ⭐⭐⭐⭐ | ⭐⭐⭐ | ⭐⭐⭐⭐ | ⭐⭐⭐⭐ | ⭐⭐⭐⭐⭐ | ⭐⭐⭐ |
| **特征稀疏** | ⭐⭐⭐⭐⭐ | ⭐⭐ | ⭐⭐⭐⭐ | ⭐⭐⭐ | ⭐⭐⭐⭐ | ⭐⭐⭐ |
| **非线性强** | ⭐ | ⭐⭐⭐⭐ | ⭐⭐⭐⭐⭐ | ⭐⭐⭐⭐⭐ | ⭐⭐⭐⭐⭐ | ⭐⭐⭐⭐⭐ |
| **噪声多** | ⭐⭐⭐⭐ | ⭐⭐⭐⭐ | ⭐⭐⭐ | ⭐⭐⭐ | ⭐⭐ | ⭐⭐⭐⭐ |
| **需要可解释** | ⭐⭐⭐⭐⭐ | ⭐⭐⭐⭐ | ⭐⭐⭐ | ⭐⭐⭐ | ⭐ | ⭐⭐⭐ |
| **训练速度要求** | ⭐⭐⭐⭐⭐ | ⭐⭐⭐ | ⭐⭐⭐⭐ | ⭐⭐⭐⭐⭐ | ⭐⭐ | ⭐⭐⭐ |

---

## 🔍 3. 不同场景最佳模型

### **表3：应用场景 vs 推荐模型**

| 应用场景 | 数据特点 | 首选模型 | 次选模型 | 理由 |
|---------|---------|---------|---------|------|
| **房价预测** | 2万样本，地理+数值 | LightGBM | XGBoost | 地理非线性强，样本量适中 |
| **用户流失预测** | 10万样本，高基数分类 | CatBoost | LightGBM | 大量用户ID、行为类别 |
| **销售预测** | 时序+季节性 | XGBoost(+lag) | LSTM | 时序特征工程后树模型足够 |
| **信用评分** | 100万样本，需可解释 | LightGBM | Logistic | 大数据，需要SHAP解释 |
| **推荐系统** | 千万级，稀疏特征 | 神经网络(Embedding) | LightGBM | 需要学习用户/物品表示 |
| **异常检测** | 不平衡数据 | IsolationForest | XGBoost | 专门的异常检测算法 |
| **图像回归** | 图像特征 | CNN | 迁移学习 | 空间结构信息 |
| **文本情感分数** | 文本数据 | Transformer | LSTM | 需要语义理解 |
| **生产线质量** | 小样本(<500) | Ridge/SVM | RandomForest | 防止过拟合 |
| **实时预测** | 低延迟要求 | 线性模型 | LightGBM | 推理速度优先 |

## 📐 4. 模型复杂度 vs 性能权衡

### **表4：性能-复杂度-速度三角**

| 模型 | 预测性能 | 训练速度 | 推理速度 | 调参难度 | 可解释性 | 综合评分 |
|------|---------|---------|---------|---------|---------|---------|
| **LightGBM** | ⭐⭐⭐⭐⭐ | ⭐⭐⭐⭐⭐ | ⭐⭐⭐⭐⭐ | ⭐⭐⭐ | ⭐⭐⭐ | **4.6** ✅ |
| **XGBoost** | ⭐⭐⭐⭐⭐ | ⭐⭐⭐⭐ | ⭐⭐⭐⭐ | ⭐⭐⭐ | ⭐⭐⭐ | **4.4** |
| **CatBoost** | ⭐⭐⭐⭐⭐ | ⭐⭐⭐ | ⭐⭐⭐⭐ | ⭐⭐⭐⭐ | ⭐⭐⭐⭐ | **4.4** |
| **HistGradientBoosting** | ⭐⭐⭐⭐ | ⭐⭐⭐⭐⭐ | ⭐⭐⭐⭐⭐ | ⭐⭐⭐⭐⭐ | ⭐⭐⭐ | **4.4** |
| **RandomForest** | ⭐⭐⭐⭐ | ⭐⭐⭐ | ⭐⭐⭐ | ⭐⭐⭐⭐⭐ | ⭐⭐⭐⭐ | **3.8** |
| **GradientBoosting** | ⭐⭐⭐⭐ | ⭐⭐ | ⭐⭐⭐⭐ | ⭐⭐⭐ | ⭐⭐⭐ | **3.4** |
| **神经网络** | ⭐⭐⭐ | ⭐⭐ | ⭐⭐⭐⭐ | ⭐⭐ | ⭐ | **2.8** |
| **Ridge/Lasso** | ⭐⭐ | ⭐⭐⭐⭐⭐ | ⭐⭐⭐⭐⭐ | ⭐⭐⭐⭐⭐ | ⭐⭐⭐⭐⭐ | **4.2** |

**评分说明**：
- 预测性能：在表格数据上的准确度
- 训练速度：达到收敛的时间
- 推理速度：单条预测延迟
- 调参难度：达到良好性能所需的调参工作量（越高越简单）
- 可解释性：特征重要性、决策路径的可读性
